In [1]:
!pip3 install numpy

In [2]:
import os
import io
import numpy as np
from pandas import DataFrame
from sklearn.utils import shuffle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [3]:
def read_files(path):
    for root, dir_names, file_names in os.walk(path):
        for path in dir_names:
            read_files(os.path.join(root, path))
    for file_name in file_names:
        if file_name not in set(['cmds']):
            file_path = os.path.join(root, file_name)
        if os.path.isfile(file_path):
            past_header, lines = False, []
            #f = open(file_path)
            f = io.open(file_path, mode="r", encoding="Latin-1")
            for line in f:
                if past_header:
                    lines.append(line)
                elif line == '\n':
                    past_header = True
            f.close()
            yield file_path, '\n'.join(lines)

In [4]:
def build_data_frame(path, classification):
    df = DataFrame({'text': [], 'class': []})
    for file_name, text in read_files(path):
        df = df.append(
            DataFrame({'text': [text], 'class': [classification]}, index=[file_name]))
    return df

In [5]:
HAM = 0
SPAM = 1

SOURCES = [
    ('data/spam',         SPAM),
    ('data/spam_2',       SPAM),
    ('data/easy_ham',     HAM),
    ('data/hard_ham',     HAM),
]

data = DataFrame({'text': [], 'class': []})
for path, classification in SOURCES:
    data = data.append(build_data_frame(path, classification))

In [6]:
data = shuffle(data)

In [7]:
train = data.iloc[:4400]
test = data.iloc[4400:]

In [8]:
count_vectorizer = CountVectorizer()
counts = count_vectorizer.fit_transform(np.asarray(data['text']))

In [9]:
classifier = MultinomialNB()
targets = np.asarray(data['class'])
classifier.fit(counts, targets)

MultinomialNB()

In [10]:
type(test['text'])

pandas.core.series.Series

In [11]:
examples = ['Free Viagra call today!', "I'm going to attend the Linux users group tomorrow."]
example_counts = count_vectorizer.transform(examples)
predictions = classifier.predict(example_counts)
predictions

array([1., 0.])

In [12]:
example_counts = count_vectorizer.transform(test['text'])
predictions = classifier.predict(example_counts)
print('Predictions: ', predictions)

Predictions:  [1. 0. 1. ... 0. 1. 1.]


In [13]:
# Accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(test['class'], predictions)
print('Accuracy: %f' % accuracy)

Accuracy: 0.990994


In [14]:
# precision tp / (tp + fp)
precision = precision_score(test['class'], predictions)
print('Precision: %f' % precision)

Precision: 0.999029


In [15]:
# recall: tp / (tp + fn)
recall = recall_score(test['class'], predictions)
print('Recall: %f' % recall)

Recall: 0.970755


In [16]:
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(test['class'], predictions)
print('F1 score: %f' % f1)

F1 score: 0.984689
